In [81]:
import pandas, re, cleantext, nltk
import pandas as pd
from itertools import chain

In [82]:
def stem(input):
  stemmer = nltk.SnowballStemmer(language="english", ignore_stopwords=True)
  for token in input:
    stemmer.stem(token)
  return input

In [84]:
import pandas as pd
import re
from itertools import chain
import nltk
from nltk import regexp_tokenize

def tokenize(raw_data):

    # Read only 'content' column of data
    usecols = ['content']
    df = pd.read_csv(raw_data, usecols = usecols)

    # Create empty list to gather lists of tokens
    # and loop through column to add words
    allWords = []

    for cell in df['content']:
        cell = regexp_tokenize(cell, pattern=r"\s|[.,;:?!\"$]", gaps=True)
        # words = cell.split()
        # allWords.append(list(words))
        # tokens = nltk.word_tokenize(cell)
        # tokensREGEX = regexp_tokenize(cell, pattern=r"\s|[.,;:?!\"$]", gaps=True)
        # allWords.append(tokensREGEX)
    
    print(df['content'][2])
    
    # Create flat list from list of lists
    # flat_list_raw = [word for inner_list in allWords for word in inner_list]
    # flat_list_final = [word for item in flat_list_raw for word in re.split(r'[().,\'"-]', item) if word.strip()]
    # flat_list_final = [word for item in flat_list_raw for word in item.split('.')]
    
    # print(len(flat_list_final))
    # for i in range(5000,5050):
    #     print(flat_list_final[i])
           
    return df

tokenize('news_sample.csv')



Never Hike Alone: A Friday the 13th Fan Film USA | 2017 | 54 min A fan tribute to Friday the 13th, Never Hike Alone follows an adventure

Never Hike Alone: A Friday the 13th Fan Film USA | 2017 | 54 min



A fan tribute to Friday the 13th, Never Hike Alone follows an adventure blogger, Kyle McLeod (Andrew Leighty), who uncovers the long lost remains of Camp Crystal Lake while on a solo backpacking trip.

Ignoring the campfire tales from his childhood, Kyle's search turns deadly when he makes the grave mistake of crossing the path of Camp Blood’s legendary mass murderer, Jason Voorhees.

Stranded in the forest with Jason on his heels, Kyle must push his survival skills to their limits if he hopes to survive the night.

Otherwise, he'll wind up as another lost victim of the cursed camp.



DIRECTED BY

VINCENTE DISANTI



SCREENPLAY BY

VINCENTE DISANTI and NATHAN MCLEOD



BASED ON CHARACTERS CREATED BY

SEAN CUNNINGHAM

VICTOR MILLER

RON KURTZ



STARRING:

ANDREW LEIGHTY as KYLE MCLE

,content
0,Sometimes the power of Christmas will make you...
1,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...
2,Never Hike Alone: A Friday the 13th Fan Film U...
3,"When a rare shark was caught, scientists were ..."
4,Donald Trump has the unnerving ability to abil...
...,...
245,"Prison for Rahm, God’s Work And Many Others\n\..."
246,4 Useful Items for Your Tiny Home\n\nHeadline:...
247,Former CIA Director Michael Hayden said Thursd...
248,Antonio Sabato Jr. says Hollywood's liberal el...


In [ ]:
# import tokenize as toke

def run():
  data = "news_sample.csv"
  tokenizedData =tokenize(data)
  stemmedData = stem(tokenizedData)
  
  for variable in stemmedData:
    print(variable)

run()

Sometimes
the
power
of
Christmas
will
make
you
do
wild
and
wonderful
things
You
do
not
need
to
believe
in
the
Holy
Trinity
to
believe
in
the
positive
power
of
doing
good
for
others
The
simple
act
of
giving
without
receiving
is
lost
on
many
of
us
these
days
as
worries
about
money
and
success
hold
us
back
from
giving
to
others
who
are
in
need
One
congregation
in
Ohio
was
moved
to
action
by
the
power
of
a
sermon
given
at
their
church
on
Christmas
Eve
The
pastor
at
Grand
Lake
United
Methodist
Church
in
Celina
Ohio
gave
an
emotional
sermon
about
the
importance
of
understanding
the
message
of
Jesus
For
many
religious
people
the
message
of
Jesus
is
to
help
others
before
yourself
to
make
sure
the
people
who
are
suffering
get
the
help
they
need
to
enjoy
life
a
little
bit
The
sermon
was
really
about
generosity
and
what
that
can
look
like
in
our
lives
Jesus
lived
a
long
time
ago
and
he
acted
generously
in
the
fashion
of
his
time
–
but
what
would
a
generous
act
look
like
in
our
times
That
was
the
